In [1]:
from zipdl.algos import multi_factor_training as mf
from datetime import datetime as dt
from zipline import run_algorithm
import pandas as pd
import numpy as np

In [2]:
sample_weights = [.5, .5]
#sample_weights = [1]
start = pd.to_datetime('2018-01-01').tz_localize('US/Eastern')
end = pd.to_datetime('2018-02-01').tz_localize('US/Eastern')
start_capital = 5000
z = utils.get_current_universe(start)

In [2]:
%%bash
zipline ingest -b quantopian-quandl

[2018-04-11 06:52:51.832443] INFO: zipline.data.bundles.quandl: Writing data to /home/austen/.zipline/data/quantopian-quandl/2018-04-11T06;52;29.146211.


In [3]:
from zipdl.utils import utils
from zipdl.data import models as m

In [4]:
utils.clean_db_time_series(mf.DB_FACTORS_USED)

cleaning Shares_Outstanding
cleaning EBITDA
cleaning Short term debt
cleaning Long Term Debt
cleaning Cash and Cash EquivalentsTotal Assets
cleaning Total Liabilities
cleaning Free Cash Flow


In [8]:
utils.clean_db_time_series(['Total Assets'])

cleaning Total Assets


In [9]:
print(utils.get_fundamental(start, 'EBITDA', 'AAPL'))
print(utils.get_fundamental(start, 'Short term debt', 'AAPL'))
print(utils.get_fundamental(start, 'Long Term Debt', 'AAPL'))
print(utils.get_fundamental(start, 'EBITDA', 'AAPL'))
print(utils.get_fundamental(start, 'Cash and Cash Equivalents', 'AAPL'))
print(utils.get_fundamental(start, 'Total Assets', 'AAPL'))
print(utils.get_fundamental(start, 'Total Liabilities', 'AAPL'))
print(utils.get_fundamental(start, 'Shares_Outstanding', 'AAPL'))

71501.0
140458.0
35867.0
71501.0
74181.0
375319.0
241272.0
5087.056


In [32]:
from zipdl.data import models as m
def get_fundamentals(date, fundamental, tickers, session=utils.session):
    data = session.query(m.Fundamentals).filter(m.Fundamentals.metric==fundamental).all()
    data = [obj for obj in data if obj.ticker in tickers]
    dict_tickers = [obj.ticker for obj in data]
    data.sort(key = lambda obj: tickers.index(obj.ticker))
    def get_close(obj, date):
        #print(obj.ticker)
        if not obj.time_series:
            return np.nan
        close = utils.find_closest_date(obj.time_series.keys(), date)
        return obj.time_series[close]
    values = [get_close(obj, date) for obj in data]
    dictionary = dict(zip(dict_tickers, values))
    return [dictionary[ticker] if ticker in dictionary else np.nan for ticker in tickers]


In [33]:
x = get_fundamentals(start, 'Shares_Outstanding', z)
x

[50.196999999999996,
 351.63699999999994,
 5.744,
 177.703,
 43.476000000000006,
 13.845,
 742.794,
 31.456,
 1956.842,
 136.299,
 62.0,
 30.079,
 79.54899999999999,
 38.551,
 20.131,
 656.7130000000001,
 130.635,
 143.734,
 69.668,
 46.926,
 687.057,
 157.023,
 389.05199999999996,
 425.483,
 130.745,
 31.461,
 778.5930000000001,
 42.153,
 154.003,
 10.52,
 165.968,
 17.531,
 48.693000000000005,
 108.24799999999999,
 3.009,
 38.701,
 7.144,
 398.157,
 29.875999999999998,
 289.087,
 296.252,
 893.7230000000001,
 64.658,
 187.021,
 77.86,
 1223.445,
 314.60400000000004,
 114.053,
 86.954,
 431.034,
 240.77599999999998,
 86.911,
 374.92800000000005,
 2724.437,
 123.045,
 281.487,
 7714.59,
 nan,
 162.991,
 787.317,
 38.141,
 34.711,
 8.971,
 28.643,
 19.999000000000002,
 7.031000000000001,
 126.385,
 1373.1960000000001,
 2.765,
 1422.1429999999998,
 nan,
 79.533,
 6.287999999999999,
 174.041,
 35.483000000000004,
 166.269,
 533.637,
 52.257,
 69.893,
 79.645,
 9.077,
 91.244,
 1.793000000

In [ ]:
import numpy as np
import pandas as pd
import pyfolio
import zipline


from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.filters import StaticAssets
from zipline.data import bundles
from zipline.api import *
from zipline.pipeline.data import USEquityPricing
import talib

from zipdl.utils import utils
import datetime as dt

# Weeks between a rebalance
REBALANCE_PERIOD = 4

# Lookback window, in days, for Momentum (Bollinger Bands and RSI) factor
MOMENTUM_LOOKBACK = 30

# If true, will switch from mean-reverting to trend-following momentum
TREND_FOLLOW = True

# Upper/lower SD's required for Bollinger Band signal
BBUPPER = 1.5
BBLOWER = 1.5

NORMALIZE_VALUE_SCORES = False

# Upper/lower scores required for RSI signal
RSI_LOWER = 30
RSI_UPPER = 70

# Percentile in range [0, 1] of stocks that are shorted/bought
SHORTS_PERCENTILE = 0.05
LONGS_PERCENTILE = 0.05

# Constraint Parameters
MAX_GROSS_LEVERAGE = 1.0

DOLLAR_NEUTRAL = False

# If True, will screen out companies that have earnings releases between rebalance periods
AVOID_EARNINGS = True

# If False, shorts won't be ordered
ALLOW_SHORTS = True

# If True, will cut positions causing losses of LOSS_THRESHOLD or more
CUT_LOSSES = False

# Positions losing this much or more as a fraction of the investment will be cut if CUT_LOSSES is True
LOSS_THRESHOLD = 0.03

# Whether or not to print pipeline output stats. For backtest speed, turn off.
PRINT_PIPE = False

#=================util=============================
bundle_data = bundles.load('quantopian-quandl')
#Throws out tickers not found in quandl-quantopian data (mostly tickers with no vol)
def safe_symbol(ticker):
    try:
        bundle_data.asset_finder.lookup_symbol(ticker, as_of_date=None)
        return ticker
    except: 
        return None
def safe_symbol_convert(tickers):
    filtered_list = list(filter(None.__ne__, [safe_symbol(ticker) for ticker in tickers]))
    assets = bundle_data.asset_finder.lookup_symbols(filtered_list, as_of_date=None)
    return assets
#==================PRIMARY==========================
def initialize_environment(weight, trading_start):
    def initialize(context):
        set_commission(commission.PerShare(cost=0.005, min_trade_cost=1.00))
        context.universe = utils.get_current_universe(trading_start)
        context.Factor_weights = weight
        context.curr_date = trading_start

        schedule_function(rebalance_portfolio, date_rules.week_start(days_offset=2), time_rules.market_open(hours=1))
        
        schedule_function(cancel_open_orders, date_rules.every_day(), time_rules.market_open())

        schedule_function(prime_pipeline, date_rules.week_start(days_offset=1), time_rules.market_close())

        context.weights = None
        context.run_pipeline = True #We want to run stock selector immediately
        context.weeks_since_rebalance = 0

        attach_pipeline(make_pipeline(context), 'my_pipeline')


    return initialize
#schedule trading monthly
#schedule stop loss/take gain daily

def handle_data(context, data):
    pass
    
def rebalance_portfolio(context, data):
    # rebalance portfolio
    close_old_positions(context, data)
    total_weight = np.sum(context.weights.abs())
    weights = context.weights / total_weight
    for stock, weight in weights.items():
        order_target_percent(stock, weight)

def before_trading_start(context, data):
    if not context.run_pipeline:
        return

    context.output = pipeline_output('my_pipeline')

    # Do some pre-work on factors
    if NORMALIZE_VALUE_SCORES:
        normalizeValueScores(context)
    
    # Rank each column of pipeline output (higher rank is better). Then create composite score based on weighted average of factor ranks
    individual_ranks = context.output.rank()
    individual_ranks *= context.Factor_weights
    ranks = individual_ranks.sum(axis=1).dropna().sort_values() + 1

    
    number_shorts = int(SHORTS_PERCENTILE*len(ranks))
    number_longs = int(LONGS_PERCENTILE*len(ranks))
    
    if number_shorts == 1 or number_longs == 1:
        number_shorts = number_longs = 0

    if (number_shorts + number_longs) > len(ranks):
        ratio = float(number_longs)/number_shorts
        number_longs = int(ratio*len(ranks)) - 1
        number_shorts = len(ranks) - number_longs
        
    shorts = 1.0 / ranks.head(number_shorts)
    shorts /= sum(shorts)
    shorts *= -1
    
    longs = ranks.tail(number_longs)
    longs /= sum(longs)
    
    if ALLOW_SHORTS:
        context.weights = shorts.append(longs)
    else:
        context.weights = longs
    print(context.weights)
    # log.info(context.weights)
        
'''    context.output = context.output.rename(columns={'momentum_score':'momentum','sentiment_score': 'sentiment', 'value_score': 'value'})
    print("WEIGHTS: \n" + str(context.weights))
    
    comp_scores = context.output * context.Factor_weights
    comp_scores = comp_scores.sum(axis=1)
    comp_scores.name = 'composite'
    scores = pd.concat([comp_scores, context.output], axis=1)
    scores = scores.fillna(value=0.0)
    scores = scores.sort_values('composite').round(4)
    print("FACTOR SCORES: \n" + str(scores.loc[context.weights.index, :]))

    ranks.name = 'ranks'
    ranked_scores = pd.concat([ranks, comp_scores, context.output], axis=1)
    ranked_scores = ranked_scores.fillna(value=0.0)
    ranked_scores = ranked_scores.sort_values('ranks')
    ranked_scores = ranked_scores.drop(['ranks'], axis=1)
    ranked_scores = ranked_scores.round(4)
    print("BOTTOM SCORES: \n" + str(ranked_scores.head(10)))
    print("TOP SCORES: \n" + str(ranked_scores.tail(10)))
    print("Number in universe" + str(len(ranked_scores)))'''
    

#==================UTILS==========================

def cancel_open_orders(context, data):
    for stock in get_open_orders():
        for order in get_open_orders(stock):
            cancel_order(order)

def close_old_positions(context, data):
    to_be_closed = pd.Series()
    for stock in context.portfolio.positions:
        if stock not in context.weights:
            to_be_closed.set_value(stock, 0.0)
            
    context.weights = to_be_closed.append(context.weights)

#===================FACTORS=========================
DB_FACTORS_USED = ['Shares_Outstanding', 'EBITDA', 'Short term debt', 'Long Term Debt', 'Cash and Cash Equivalents',
                    'Total Assets', 'Total Liabilities', 'Free Cash Flow']
class ValueFactor(CustomFactor):
    """
    For every stock, computes a value score for it, defined as the product of 
    its book-price, FCF-price, and EBITDA-EV ratio, where a higher value is 
    desirable.
    """
    inputs = [USEquityPricing.close]
    window_length = 1
    
    def compute(self, today, asset_ids, out, close):
        #for verification
        tickers = [asset.symbol for asset in bundle_data.asset_finder.retrieve_all(asset_ids)]
        #print(tickers)
        
        shares_outstanding = np.array([get_fundamentals(today, 'Shares_Outstanding', tickers)])
        market_cap = close * np.array([shares_outstanding])
        #ev_to_ebitda
        ebitda = np.array([get_fundamentals(today, 'EBITDA', tickers)])
        shortTermDebt = np.array([get_fundamentals(today, 'Short term debt', tickers)])
        longTermDebt = np.array([get_fundamentals(today, 'Long Term Debt', tickers)])
        c_and_c_equivs = np.array([get_fundamentals(today, 'Cash and Cash Equivalents', tickers)])
        ev = shortTermDebt + longTermDebt + market_cap - c_and_c_equivs
        ebitda_to_ev = ebitda / ev
        
        #Book to price
        totalAssets = np.array([get_fundamentals(today, 'Total Assets', tickers)])
        totalLiab = np.array([get_fundamentals(today, 'Total Liabilities', tickers)])
        book_to_price = (totalAssets - totalLiab) / shares_outstanding

        fcf = np.array([get_fundamentals(today, 'Free Cash Flow', tickers)])
        fcf_yield = fcf / close

        values = ebitda_to_ev * book_to_price * fcf_yield

        out[:] = values

class MomentumFactor(CustomFactor):
    """
    Uses Bollinger Bands and RSI measures to determine whether or not a stock 
    should be bought (return 1), sold (return -1), or if there is no signal 
    (return 0). For a signal, both metrics have to indicate the same signal 
    (e.g., price below lower Bollinger Band and RSI below RSI_LOWER)
    """
    window_length = MOMENTUM_LOOKBACK+10
    inputs = [USEquityPricing.close]
    
    def compute(self, today, asset_ids, out, close):
        for i in range(len(out)):
            out[i] = 0
            prices = close[:, i]
            try:
                upperBand, middleBand, lowerBand = talib.BBANDS(
                    prices,
                    timeperiod = MOMENTUM_LOOKBACK,
                    nbdevup=BBUPPER,
                    nbdevdn=BBLOWER,
                    matype=0)
                
                rsi = talib.RSI(prices, timeperiod=MOMENTUM_LOOKBACK)
                               
            except:
                out[i] = 0
                continue
            
            out[i] = 0
            if rsi[-1] < RSI_LOWER:
                out[i] += .5
            elif rsi[-1] > RSI_UPPER:
                out[i] -= .5
            if prices[-1] < lowerBand[-1]:
                out[i] += .5
            elif prices[-1] > upperBand[-1]:
                out[i] -= .5
                
            if TREND_FOLLOW:
                out[i] *= -1
#============================Pipeline Stuff=============================
def make_pipeline(context):
    tradable_assets = safe_symbol_convert(context.universe)
    screen = StaticAssets(tradable_assets)
    context.universe = tradable_assets

    value_factor = ValueFactor()
    momentum_factor = MomentumFactor()
    pipe = Pipeline(
        columns = {
            'value_score': value_factor,
            'momentum_score': momentum_factor,
        }
    )
    pipe.set_screen(screen)
    return pipe
def prime_pipeline(context, data):
    context.weeks_since_rebalance += 1
    if context.weeks_since_rebalance >= REBALANCE_PERIOD:
        context.run_pipeline = True
        context.weeks_since_rebalance = 0



In [ ]:
initialize = initialize_environment(sample_weights, start)
perf = run_algorithm(start, end, 
                    initialize=initialize, 
                    capital_base=start_capital, 
                    before_trading_start=before_trading_start)

In [ ]:
for ticker in tickers:
    print(ticker)
    utils.get_fund

[utils.get_fundamental(today, 'Shares_Outstanding', ticker) for ticker in tickers]

In [10]:
z = utils.get_current_universe(start)
len(z)

1199

In [11]:
'AA' in z

False

In [5]:
import quandl
import os
quandl.ApiConfig.api_key = os.environ['QUANDL_KEY']

In [38]:
start_str = '2015-02-03, 2015-02-04, 2015-02-05'
counter = 0
n_a = []
for ticker in z:
    if quandl.get_table('WIKI/PRICES', ticker=ticker.replace(' ', ''), date=start_str).empty:
        print(ticker)
        counter += 1
        n_a.append(ticker)

USAT
CYAN
RWC
CGA
AGI
FKWL
EEI
NERV
TRNS
ALV
ZHNE
FORD
AMRK
SPAN
DDMG
FONR
CIK
BRID
AKER
ITI
HNNA
NSHI
FRD
C208
RYAM
AMDA
CPXX
MGCD
IHT
MTII
TTLO
FDC
PRGB
SEV
EEP
ESP
GLPI
BSTG
BOOT
BLUD
RICK
LRDC
SMED
SOHU
WPC
MAYS
TSQ
QVC
RMCF
SINO
KLIC
JAZZ
CDNA
VALU
CLRB
NSPR
BGSF
DNOW
PSEG
SMID
SKY
VERU
DYNT
CNVR
SRNE
CIK0
VIRC
DTRM
GLGI
EFH
SAGE
ZYXI
ACPW
CCS
CETX
TSSI
ADPT
LPTH
INFOR
MAMS
SRT
AWRE
HYH
NHTC
NPS
ON
HDSN
VRS
IMSCQ
IDSY
GPRO
TST
SMSI
CK00015503
IVDN
SPGI
GENC
LTRX
AAXN
PMD
CATS
LEATPK
OLIN
AIRI
SMIT
HGEN
PPSI
CENT
AR
C214
TMED
PERF
JWA
W
AGHI
NLSNNV
WNDW
CMD
INFU
C730
CCFI
NLST
ASNB
UA
TPCS
CHKE
LOGI
NSSC
PSDV
NAII
TBTC
OGES
WSCI
SIMU
GTLT
SHPGF
WINH
ERS
TSRI
ADAT
SOHO
RFMD
MIC
KTEC
PSTI
CNNX
ACT
TA
BMRA
DGTC
MTBC
SCND
NCSO
CAGU
KE
CPAH
BFB
SRC
NGL
SLNM
PSSR
BVSN
LULU
PRTX
KATE
GIDYL
UG
LEDS
CLNT
MHH
BLFS
IIN
FLEX
AJRD
TSE
AEHR
ANET
TLCC
STAY
FLUG
CRWS
ELMD
TAYD
CIK0001446
EPD
JVA
BX
CSBR
IEHC
BRX
AMOT
IAIC
ETEC
ABCD
ENG
ALT
SOFO
NEPH
ABTL
AMH
DDE
BRRE
C099
ABMC
SOTK
OCC
DPLO
AHPI
B

In [39]:
len(n_a)

243